In [1]:
import os
import sys
import json
from dotenv import load_dotenv
load_dotenv()

# Add the parent directory
parent_dir = os.path.dirname(os.getcwd())
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

# Kentro

### GetPaySlipTool

In [2]:
from tools.GetPayslipTool import get_payslip
candidate_id = os.environ.get("KENTRO_TEST_CANDIDATE_ID")

In [3]:
payslips = get_payslip.invoke({
    "candidate_id": candidate_id, 
    "latest": False
})

In [4]:
payslips

[Payslip(id=1209941, business_unit_id=10023, serial_number='25-45-001', frequency='W', entry_date='2025-11-12T00:11:06', amount=404.99),
 Payslip(id=1194574, business_unit_id=10023, serial_number='25-44-001', frequency='W', entry_date='2025-11-04T22:05:43', amount=383.36),
 Payslip(id=1180618, business_unit_id=10023, serial_number='25-43-001', frequency='W', entry_date='2025-10-28T21:52:21', amount=223.92),
 Payslip(id=1167313, business_unit_id=10023, serial_number='25-42-001', frequency='W', entry_date='2025-10-21T21:43:20', amount=211.32),
 Payslip(id=1154330, business_unit_id=10023, serial_number='25-41-001', frequency='W', entry_date='2025-10-14T21:56:35', amount=561.07),
 Payslip(id=1139129, business_unit_id=10023, serial_number='25-40-001', frequency='W', entry_date='2025-10-07T21:56:10', amount=457.63),
 Payslip(id=1125205, business_unit_id=10023, serial_number='25-39-001', frequency='W', entry_date='2025-09-30T21:46:22', amount=453.81),
 Payslip(id=1112483, business_unit_id=100

In [5]:
# Print details of the first payslip
item = payslips[0]
print("\n")
print(f"id: {item.id}") 
print(f"frequency: {item.frequency}")
print(f"amount: {item.amount}")



id: 1209941
frequency: W
amount: 404.99


In [6]:
latest = get_payslip.invoke({
    "candidate_id": candidate_id, 
    "latest": True
})

In [7]:
latest.entry_date

'2025-11-12T00:11:06'

### GetReservationsTool

In [8]:
from tools.GetReservationsTool import get_reservations

In [9]:
reservations = get_reservations.invoke({
    "candidate_id": candidate_id})

reservations

[ReservationBalance(candidate_id=100102195, business_unit_tax_category_id=10269, type_id=10064, description='Vakantieuren wettelijk lopend jaar', setu_code=None, display_in_portal=False, year=2025, accrued_amount=4.31, accrued_unit='U', available_balance=None, withdrawal_unit=None, max_withdrawal_limit=0.0, allow_withdraw_all=False),
 ReservationBalance(candidate_id=100102195, business_unit_tax_category_id=10269, type_id=10063, description='Vakantieuren bovenwettelijk', setu_code=None, display_in_portal=False, year=2025, accrued_amount=10.26, accrued_unit='U', available_balance=10.26, withdrawal_unit='U', max_withdrawal_limit=0.0, allow_withdraw_all=True),
 ReservationBalance(candidate_id=100102195, business_unit_tax_category_id=10269, type_id=10019, description='Vakantiegeld', setu_code=None, display_in_portal=False, year=2025, accrued_amount=167.59, accrued_unit='G', available_balance=167.59, withdrawal_unit='G', max_withdrawal_limit=0.0, allow_withdraw_all=True),
 ReservationBalance

In [10]:
reservations[0].description

'Vakantieuren wettelijk lopend jaar'

### GetContractsTool

In [11]:
from tools.GetContractsTool import get_contracts

In [12]:
contracts = get_contracts.invoke({
    "candidate_id": candidate_id})

contracts

[Contract(id=131594, code='00087783', status='Gesloten', start_date='2025-02-10T00:00:00', end_date='2025-03-09T00:00:00', function_description='Warehouse Operative', hourly_wage=14.7, wage_unit='Uur', employment_desc='Normaal'),
 Contract(id=137146, code='00094529', status='Gesloten', start_date='2025-03-10T00:00:00', end_date='2025-06-15T00:00:00', function_description='Warehouse Operative', hourly_wage=14.7, wage_unit='Uur', employment_desc='Normaal'),
 Contract(id=158187, code='00117938', status='Lopend', start_date='2025-06-16T00:00:00', end_date='2025-12-14T00:00:00', function_description='Warehouse Operative', hourly_wage=14.7, wage_unit='Uur', employment_desc='Normaal')]

In [13]:
print(f"start_date: {contracts[0].start_date}")
print(f"end_date: {contracts[0].end_date}")
print(f"status: {contracts[0].status}")
print(f"hourly_wage: {contracts[0].hourly_wage}")

start_date: 2025-02-10T00:00:00
end_date: 2025-03-09T00:00:00
status: Gesloten
hourly_wage: 14.7


# Planbition

### GetScheduleTool

In [14]:
from tools.GetScheduleTool import get_schedule

In [15]:
TEST_EMPLOYEE_NUMBER = "RE0036443" 
TEST_START_DATE = "2025-05-01"
TEST_END_DATE = "2025-05-31"

schedule = get_schedule.invoke({
            "employee_number": TEST_EMPLOYEE_NUMBER,
            "start_date": TEST_START_DATE,
            "end_date": TEST_END_DATE
        })

print(f"Success! Found {len(schedule)} schedule items.")

Success! Found 7 schedule items.


In [16]:
for item in schedule[0]:
    print(item)

('id', 'DEF894FC-1A82-493D-AE67-7FD0C2E94177_627B699E-0FA6-4153-AC90-C32EBFC230DD_280520250730')
('employeeNumber', 'RE0036443')
('personId', 'RE0036443')
('organisationStructureId', 'Bleckmann Nederland - Nijverheidsstraat')
('organisationStructureName', 'Bleckmann Nederland - Nijverheidsstraat')
('customerId', '0FDD57AF-5DA2-432A-B95F-C158CEBC822A')
('customerName', 'Bleckmann Nederland - Nijverheidsstraat')
('customerStructureName', 'Ruud')
('shiftId', 'F40D2C95-433D-4446-A200-E3A6D5E1A215')
('shiftName', 'Ruud Reco')
('shiftQualificationDemandId', '627B699E-0FA6-4153-AC90-C32EBFC230DD')
('shiftQualificationDemandName', 'Ruud Reco')
('scheduleEmployeeAdministrativeComponents', [])
('startTime', {'year': 2025, 'month': 5, 'day': 28, 'hour': 7, 'minute': 30, 'second': 0, 'millisecond': 0})
('endTime', {'year': 2025, 'month': 5, 'day': 28, 'hour': 16, 'minute': 30, 'second': 0, 'millisecond': 0})
('plannerRemarkPublic', None)
('plannerRemarkPrivate', None)
('employeeRemark', None)
('is

In [17]:
schedule[0].customerName

'Bleckmann Nederland - Nijverheidsstraat'